In [2]:
# Importing libraries
import time, os
from selenium import webdriver
from PIL import Image

In [5]:
def traffic_screenshot(ref, lat, long, zoom=15):
    ##### Generate a cropped screenshot of the immediate vicinty of coordinates ####

# Gets the path to the right chromedriver
    path = 'C:\Windows\chromedriver'
# Creating aurl from a list of coordinates
    link = 'https://www.google.com/maps/@{},{},{}z/data=!5m1!1e1'.format(lat, long, zoom)

# links = {'Kampala':'https://www.google.com/maps/dir/Entebbe/Kajjansi/@0.3206818,32.5481476,13.04z/data=!4m14!4m13!1m5!1m1!1s0x177d86b753c20ab3:0xa4a550c375cc2c14!2m2!1d32.463708!2d0.0511839!1m5!1m1!1s0x177d99d8aceb164f:0xb3b42975dfc7a0fa!2m2!1d32.5401025!2d0.2070254!3e0!5m1!1e1',
#         'Entebbe_Kamapla':'http://www.google.com/maps/dir/Kololo,+Kampala/Makerere+University,+University+Rd,+Kampala/@0.1472201,32.4932571,12z/data=!4m18!4m17!1m5!1m1!1s0x177dbba2a2d9eead:0x698c319a2299b891!2m2!1d32.5949199!2d0.327297!1m5!1m1!1s0x177dbb6d88e54def:0xddc6fcfbe10b089d!2m2!1d32.5710773!2d0.3292819!2m3!6e0!7e2!8j1585202400!3e0!5m1!1e1',
#         'Jinja_Kampala': 'https://www.google.com/maps/dir/Kololo,+Kampala/Makerere+University,+University+Rd,+Kampala/@0.3465628,32.8439372,11.48z/data=!4m18!4m17!1m5!1m1!1s0x177dbba2a2d9eead:0x698c319a2299b891!2m2!1d32.5949199!2d0.327297!1m5!1m1!1s0x177dbb6d88e54def:0xddc6fcfbe10b089d!2m2!1d32.5710773!2d0.3292819!2m3!6e0!7e2!8j1585202400!3e0!5m1!1e1'
#         }
# https://www.google.com/maps/@0.1472201,32.4932571,12z/data=!5m1!1e1

    with webdriver.Chrome(path) as driver:
        # ensuring the protocol is attached, this allows a www..... url to be added
        if ((('https://') in link) | (('http://') in link)):
            linkWithProtocol = link
        else:
            linkWithProtocol = 'http://' + str(link)
        # these values represent the sizes of the entire browser window and not the viewport.
        features = {'output': str(ref) +'.png',
                'width': 2200,
                'height': 1800}

        # set the window size
        driver.set_window_size(features['width'], features['height'])
        driver.get(linkWithProtocol)
        time.sleep(8)
        
        driver.save_screenshot(features['output'])
        
        im = Image.open(features['output']) # uses PIL library to open image in memory

        left = 550
        top= 50
        right = 1925
        bottom = 939
        
        im1 = im.crop((left, top, right, bottom)) # defines crop points starting top left
#         im1.show()  

        timestr = time.strftime("%Y-%m-%d %H_%M_%S")
        im1.save( str(ref) +'_' +str(timestr) +".png")


In [7]:
# Create a dict of all location references and coordinates in addition to the three named here
locations_dict ={'Kampala': [0.3206818,32.5481476,13.04], 
                 'Entebbe_Kampala': [0.1472201,32.4932571,12],
                 'Jinja_Kampala': [0.3465628,32.8439372,11.48]
                }

# locations_dict ={'loc_01': [0.2519447,32.5139749], 'loc_02': [0.3605524,32.7389841]}

for key, value in locations_dict.items():
    traffic_screenshot(key, value[0], value[1], value[2])

# Reading in the images - this would ideally be done in the cloud and saved to BQ table

In [ ]:
# Import required libraries
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os, re, glob, ast, cv2
from matplotlib import pyplot as plt
import pandas as pd


%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = 999
#To show full length of values
pd.options.display.max_columns = 999

In [ ]:
def traffic_level(image):
    
    ### READS IN IMAGE AND COUNTS THE NUMBER OF GREEN, ORANGE, RED BURGUNDY PIXELS
    ### SAVES AS PNG WITH LOCATION AND DATETIME ####
    
    
    # # RED ON TRAFFIC IS RGB[231, 63, 34] or HSV [4,217,231]
    # # ORANGE ON TRAFFIC IS [247, 151, 64] or HSV [ 14, 189, 247]
    # # GREEN ON TRAFFIC IS RGB [119, 213,99] or HSV  [ 55, 136, 213]
    # # BURGINDY ON TRAFFIC IS RGB [123,33,25]  or HSV [2, 203, 123]
    
    cols = {'green': [ 55, 136, 213], 'orange': [ 14, 189, 247], 'red': [4,217,231], 'burgundy':[2, 203, 123]}
    color_counts = []
    # Load an color image in colour
    image = cv2.imread(image,1)
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for col in cols:
        lower_diff = np.array([-2, -10, -10])
        lower = cols[col] + lower_diff
        higher_diff = np.array([2, 10, 8])
        higher = cols[col] + higher_diff
    
        # Threshold the HSV image to get only red colors
        mask = cv2.inRange(hsv, lower, higher) 
        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image,image, mask= mask)
    
        # converting image into a 2d arry of lenth the area of pixels and width three values
        modified_image = res.reshape(res.shape[0]*res.shape[1], 3)
        if np.count_nonzero(modified_image) == 0:
            color_pixels = 0
        else:
            # modified_image = cv2.cvtColor(modified_image, cv2.COLOR_BGR2RGB)
        
            # Using kmeans to identify each color
        
            number_of_colors = 2
            clf = KMeans(n_clusters = number_of_colors)
            labels = clf.fit_predict(modified_image)
            counts = Counter(labels)
            # Identifying the color at the centre of the cluster
            center_colors = clf.cluster_centers_
            # We get ordered colors by iterating through the keys ie 0,1 in this case
            ordered_colors = [center_colors[i] for i in counts.keys()]
            rgb_colors = [ordered_colors[i] for i in counts.keys()]
            color_pixels = counts[1]
#         print('For image ' + str(infile) +', The total number of ' +str(col) +' pixels is: ', color_pixels)
        color_counts.append(color_pixels)
    return color_counts;

In [ ]:
# Running script on all images in folder
df = pd.DataFrame(columns=['image', 'counts'])

for map_image in glob.glob('*.png')[:1]:
    color_counts = traffic_level(map_image)
    pairs = [map_image, color_counts]
    print(pairs)
    pairs_series = pd.Series(pairs, index=df.columns)
    df = df.append(pairs_series, ignore_index=True)
df